Using SQLAlchemy to Talk to a Database
=====================
SqlAlchemy helps you use a database to store and retrieve information from python.  It abstracts the specific storage engine from te way you use it - so it doesn't care if you end up using MySQL, SQLite, or whatever else. In addition, you can use core and the object-relational mapper (ORM) to avoid writing any SQL at all.  The [SQLAlchemy homepage](http://www.sqlalchemy.org/) has lots of good examples and full documentation.

In [2]:
from sqlalchemy import *
import datetime

## Basic SQL Generation
The core library generates SQL for you.  Read more about it on their [expression language tutorial page](http://docs.sqlalchemy.org/en/rel_1_0/core/index.html). Below are some basic examples.

### Creating a Table
Read more about [defining and creating tables](http://docs.sqlalchemy.org/en/rel_1_0/core/tutorial.html#define-and-create-tables).

In [3]:
# add `echo=True` to see log statements of all the SQL that is generated
engine = create_engine('sqlite:///:memory:',echo=True) # just save the db in memory for now (ie. not on disk)
metadata = MetaData()
# define a table to use
queries = Table('queries', metadata,
    Column('id', Integer, primary_key=True),
    Column('keywords', String(400), nullable=False),
    Column('timestamp', DateTime, default=datetime.datetime.now),
)
metadata.create_all(engine) # and create the tables in the database

2017-12-05 10:45:29,113 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2017-12-05 10:45:29,114 INFO sqlalchemy.engine.base.Engine ()
2017-12-05 10:45:29,116 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2017-12-05 10:45:29,117 INFO sqlalchemy.engine.base.Engine ()
2017-12-05 10:45:29,119 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("queries")
2017-12-05 10:45:29,119 INFO sqlalchemy.engine.base.Engine ()
2017-12-05 10:45:29,121 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE queries (
	id INTEGER NOT NULL, 
	keywords VARCHAR(400) NOT NULL, 
	timestamp DATETIME, 
	PRIMARY KEY (id)
)


2017-12-05 10:45:29,122 INFO sqlalchemy.engine.base.Engine ()
2017-12-05 10:45:29,123 INFO sqlalchemy.engine.base.Engine COMMIT


### Inserting Data
Read more about generating [SQL insert statements](http://docs.sqlalchemy.org/en/rel_1_0/core/tutorial.html#insert-expressions).

In [4]:
insert_stmt = queries.insert()
str(insert_stmt) # see an example of what this will do

'INSERT INTO queries (id, keywords, timestamp) VALUES (:id, :keywords, :timestamp)'

In [5]:
import datetime
data = [
    {'keywords':'puppies', 'timestamp': datetime.date(2008, 12, 6) },
    {'keywords':'kittens', 'timestamp': datetime.date(2009, 12, 6) },
    {'keywords':'babies', 'timestamp': datetime.date(2010, 12, 6) },
]

In [7]:
db_conn = engine.connect()
for item in data:
    insert_stmt = queries.insert().values(item)
    result = db_conn.execute(insert_stmt)
    print(result.inserted_primary_key) # print out the primary key it was assigned

2017-12-05 10:46:25,160 INFO sqlalchemy.engine.base.Engine INSERT INTO queries (keywords, timestamp) VALUES (?, ?)
2017-12-05 10:46:25,161 INFO sqlalchemy.engine.base.Engine ('puppies', '2008-12-06 00:00:00.000000')
2017-12-05 10:46:25,162 INFO sqlalchemy.engine.base.Engine COMMIT
[1]
2017-12-05 10:46:25,164 INFO sqlalchemy.engine.base.Engine INSERT INTO queries (keywords, timestamp) VALUES (?, ?)
2017-12-05 10:46:25,165 INFO sqlalchemy.engine.base.Engine ('kittens', '2009-12-06 00:00:00.000000')
2017-12-05 10:46:25,166 INFO sqlalchemy.engine.base.Engine COMMIT
[2]
2017-12-05 10:46:25,168 INFO sqlalchemy.engine.base.Engine INSERT INTO queries (keywords, timestamp) VALUES (?, ?)
2017-12-05 10:46:25,169 INFO sqlalchemy.engine.base.Engine ('babies', '2010-12-06 00:00:00.000000')
2017-12-05 10:46:25,170 INFO sqlalchemy.engine.base.Engine COMMIT
[3]


In [ ]:
insert_stmt = queries.insert().values(keywords="kittens")
result = db_conn.execute(insert_stmt)
result.inserted_primary_key # print out the primary key it was assigned

### Retrieving Data
Read more about using [SQL select statments](http://docs.sqlalchemy.org/en/rel_1_0/core/tutorial.html#selecting).

In [9]:
from sqlalchemy.sql import select
select_stmt = select([queries])
results = db_conn.execute(select_stmt)
for row in results:
    print(row)

2017-12-05 10:47:14,398 INFO sqlalchemy.engine.base.Engine SELECT queries.id, queries.keywords, queries.timestamp 
FROM queries
2017-12-05 10:47:14,401 INFO sqlalchemy.engine.base.Engine ()
(1, 'puppies', datetime.datetime(2008, 12, 6, 0, 0))
(2, 'kittens', datetime.datetime(2009, 12, 6, 0, 0))
(3, 'babies', datetime.datetime(2010, 12, 6, 0, 0))


In [ ]:
select_stmt = select([queries]).where(queries.c.id==1)
for row in db_conn.execute(select_stmt):
    print row

In [ ]:
select_stmt = select([queries]).where(queries.c.keywords.like('p%'))
for row in db_conn.execute(select_stmt):
    print row

## ORM
You can use their ORM library to handle the translation into full-fledged python objects.  This can help you build the Model for you [MVC](https://en.wikipedia.org/wiki/Model–view–controller) solution.

In [ ]:
import datetime
from sqlalchemy import *
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
Base = declarative_base()

### Creating a class mapping
Read more about [creating a mapping](http://docs.sqlalchemy.org/en/rel_1_0/orm/tutorial.html#declare-a-mapping).

In [ ]:
class Query(Base):
    __tablename__ = 'queries'
    id = Column(Integer, primary_key=True)
    keywords = Column(String(400))
    timestamp = Column(DateTime,default=datetime.datetime.now)
    def __repr__(self):
        return "<Query(keywords='%s')>" % (self.keywords)
Query.__table__

### Creating a connection and session
Read more about [creating this stuff](http://docs.sqlalchemy.org/en/rel_1_0/orm/tutorial.html#creating-a-session).

In [ ]:
engine = create_engine('sqlite:///:memory:') # just save the db in memory for now (ie. not on disk)
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)
my_session = Session()

### Inserting Data
Read more about [inserting data with an ORM](http://docs.sqlalchemy.org/en/rel_1_0/orm/tutorial.html#adding-new-objects).

In [ ]:
query = Query(keywords="iguana")
query.keywords

In [ ]:
my_session.add(query)
my_session.commit()
query.id

In [ ]:
query

### Retrieving Data
Read more about [retrieving data from the db](http://docs.sqlalchemy.org/en/rel_1_0/orm/tutorial.html#querying) via an ORM class.

In [ ]:
for q in my_session.query(Query).order_by(Query.timestamp):
    print q

In [ ]:
query1 = Query(keywords="robot")
query2 = Query(keywords="puppy")
my_session.add_all([query1,query2])
my_session.commit()

In [ ]:
for q in my_session.query(Query).order_by(Query.timestamp):
    print q

In [ ]:
for q in my_session.query(Query).filter(Query.keywords.like('r%')):
    print q